In [1]:
# YOLOv5 설치
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

C:\workspace_multi07\web\15_final project\yolov5


Cloning into 'yolov5'...


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
     -------------------------------------- 41.2/41.2 kB 967.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 1.1 MB/s eta 0:00:36
   ---------------------------------------- 0.1/38.8 MB 1.1 MB/s eta 0:00:35
   ---------------------------------------- 0.1/38.8 MB 1.1 MB/s eta 0:00:35
   ---------------------------------------- 0.1/38.8 MB 599.1 kB/s eta 0:01:05
   ---------------------------------------- 0.2/38.8 MB 787.7 kB/s eta 0:00:50
   ---------------------------------------- 0.2/38.8 MB 942.1 kB/s eta 0:00:41
   ---------------------------------------- 0.3/38.8 MB 780.5 kB/s eta 0:00:50
   ---------------------------------------- 0.4/38.8 MB 955.7 kB/s eta 0:00:41
    --------------------------------------- 0.5/38.8 MB 1.1 MB/s eta 0:00:

ERROR: Exception:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\http\client.py", line 473, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^

In [ ]:
train: data_eggplant/train
val: data_eggplant/validation

nc: 2  # 클래스 수 (예: 0 = 정상, 1 = 질병)
names: ['normal', 'disease']  # 클래스 이름

In [6]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 640.0 kB/s eta 0:01:01
   ---------------------------------------- 0.0/38.8 MB 640.0 kB/s eta 0:01:01
   ---------------------------------------- 0.0/38.8 MB 640.0 kB/s eta 0:01:01
   ---------------------------------------- 0.0/38.8 MB 178.6 kB/s eta 0:03:38
   ---------------------------------------- 0.1/38.8 MB 348.6 kB/s eta 0:01:52
   ---------------------------------------- 0.1/38.8 MB 409.6 kB/s eta 0:01:35
   ---------------------------------------- 0.1/38.8 MB 409.6 kB/s eta 0:01:35
   ---------------------------------------- 0.2/38.8 MB 398.2 kB/s eta 0:01:38
   ---------------------------------------- 0.2/38.8 MB 528.4 kB/s eta 0:01:14
   ---------------------------------------- 0.3/38.8 MB 610.3

In [16]:
import json
import os

# JSON 파일 경로
json_path = '../eggplant/train/질병/V006_77_1_15_08_03_11_1_1096q_20201008_1.JPG.json'

# JSON 파일 읽기
if os.path.exists(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        print(data)
else:
    print(f"파일이 존재하지 않습니다: {json_path}")

{'description': {'image': 'V006_77_1_15_08_03_11_1_1096q_20201008_1.JPG', 'date': '2020/10/08', 'worker': '', 'height': 2832, 'width': 4256, 'task': 77, 'type': 1, 'region': 2}, 'annotations': {'disease': 15, 'crop': 8, 'area': 3, 'grow': 11, 'risk': 1, 'points': [{'xtl': 1132, 'ytl': 561, 'xbr': 3228, 'ybr': 2697}]}}


In [2]:
import os
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2

# 데이터 경로 설정
base_dir = 'eggplant'
train_images_dir = os.path.join(base_dir, 'train', '원천 데이터')
train_labels_dir = os.path.join(base_dir, 'train', '라벨링 데이터')
validation_images_dir = os.path.join(base_dir, 'validation', '원천 데이터')
validation_labels_dir = os.path.join(base_dir, 'validation', '라벨링 데이터')

# 라벨 데이터 읽기 함수
def load_labels(labels_dir):
    labels = []
    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.json'):
            with open(os.path.join(labels_dir, label_file), 'r') as f:
                label_data = json.load(f)
                labels.append(label_data)
    return labels

train_labels = load_labels(train_labels_dir)
validation_labels = load_labels(validation_labels_dir)

# 이미지 데이터와 라벨 매핑
def load_data(images_dir, labels):
    images = []
    labels_list = []
    for label in labels:
        img_path = os.path.join(images_dir, label['description']['image'])
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            points = label['annotations']['points'][0]
            img_cropped = img[points['ytl']:points['ybr'], points['xtl']:points['xbr']]
            img_resized = cv2.resize(img_cropped, (150, 150))
            images.append(img_resized)
            labels_list.append(label['annotations']['disease'])
    images = np.array(images) / 255.0
    labels_list = to_categorical(labels_list)
    return images, labels_list

# 학습 데이터와 검증 데이터 로드
train_images, train_labels = load_data(train_images_dir, train_labels)
validation_images, validation_labels = load_data(validation_images_dir, validation_labels)

# 모델 구축
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(np.argmax(train_labels, axis=1))), activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_images, train_labels,
    epochs=30,
    batch_size=20,
    validation_data=(validation_images, validation_labels))

# 모델 저장
model.save('eggplant_disease_model.h5')

# 모델 평가
loss, accuracy = model.evaluate(validation_images, validation_labels)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

# 질병 코드 매핑
disease_mapping = {index: disease for index, disease in enumerate(np.unique(np.argmax(train_labels, axis=1)))}

# 이미지 예측 함수
def predict_image(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_disease = disease_mapping[predicted_class]
    return predicted_disease

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
# 테스트 예측
test_image_path = 'path/to/test/image.jpg'
predicted_disease = predict_image(test_image_path, model)
print(f'Predicted Disease: {predicted_disease}')